In [5]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [6]:
# 1. Parameter dasar
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 32
TRAIN_PATH = "dataset\Training\Training" 
TEST_PATH = "dataset\Testing\Testing" 

In [7]:
# 2. Preprocessing & Augmentasi
train_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    zoom_range=0.2,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    color_mode='grayscale'
)

validation_generator = train_datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    color_mode='grayscale'
)

test_generator = test_datagen.flow_from_directory(
    TEST_PATH,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    color_mode='grayscale'
)


Found 22619 images belonging to 6 classes.
Found 5654 images belonging to 6 classes.
Found 7067 images belonging to 6 classes.


In [8]:
# 3. Bangun CNN Model
model = Sequential()
model.add(Input(shape=(IMG_HEIGHT, IMG_WIDTH, 1)))
model.add(Conv2D(32, kernel_size=(3,3), activation='relu'))
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(128, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_generator.num_classes, activation='softmax'))

In [9]:
# 4. Compile Model
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [10]:
# 5. Callback
checkpoint = ModelCheckpoint('cnn_model_best.weights.h5', save_best_only=True, save_weights_only=True)
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [12]:
# 6. Train Model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=100,
    callbacks=[checkpoint, early_stop]
)
# Simpan seluruh model (struktur + bobot)
model.save("cnn_model_final2.h5")


Epoch 1/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 83s 118ms/step - accuracy: 0.4764 - loss: 1.3397 - val_accuracy: 0.4972 - val_loss: 1.3026
Epoch 2/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 91s 128ms/step - accuracy: 0.4876 - loss: 1.3201 - val_accuracy: 0.5131 - val_loss: 1.2660
Epoch 3/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 83s 118ms/step - accuracy: 0.5005 - loss: 1.2886 - val_accuracy: 0.5265 - val_loss: 1.2462
Epoch 4/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 112s 159ms/step - accuracy: 0.5131 - loss: 1.2473 - val_accuracy: 0.5318 - val_loss: 1.2261
Epoch 5/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 140s 156ms/step - accuracy: 0.5189 - loss: 1.2387 - val_accuracy: 0.5396 - val_loss: 1.1957
Epoch 6/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 86s 122ms/step - accuracy: 0.5315 - loss: 1.2082 - val_accuracy: 0.5462 - val_loss: 1.1935
Epoch 7/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 82s 115ms/step - accuracy: 0.5418 - loss: 1.1845 - val_accuracy: 0.5417 - val_loss: 1.1861
Epoch 8/100
707/707 ━━━━━━━━━━━━━━━━━━━━ 85s 121ms/step - accuracy: 0.5447

In [13]:
from tensorflow.keras.models import load_model

In [15]:


model = load_model("cnn_model_final2.h5")

# Evaluasi model pada data test
test_loss, test_accuracy = model.evaluate(test_generator)

print("Test Accuracy:", test_accuracy)
print("Test Loss:", test_loss)

221/221 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - accuracy: 0.6116 - loss: 1.0310
Test Accuracy: 0.6347813606262207
Test Loss: 0.9841932058334351


In [1]:
# 7. Visualisasi Akurasi
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

NameError: name 'plt' is not defined